In [8]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from bs4 import BeautifulSoup
from tqdm import tqdm


[nltk_data] Downloading package punkt to C:\Users\Shipra
[nltk_data]     Dureja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Shipra
[nltk_data]     Dureja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Change master CSV path in order to read the html files

In [ ]:
master_csv_path = "C:\\Users\\Shipra Dureja\\Downloads\\dataset\\master996.csv"

In [10]:
def clean(data):
    review = re.sub('[^a-zA-Z]', ' ', data)
    review = review.lower()
    review = review.split()
    all_stopwords = stopwords.words('english')
    review = [word for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    return review

In [11]:
master_data = pd.read_csv(master_csv_path,delimiter=';',encoding='unicode_escape')
dataset = []
for i, r in tqdm(master_data.iterrows()):
    name = r['Book_Name']
    author = r['Author_Name']
    book_id = r['book_id']
    genre = r['guten_genre']
    with open("C:\\Users\\Shipra Dureja\\Downloads\\dataset\\books\\{}-content.html".format(book_id.split('.epub')[0]), 'r',encoding="utf8") as file:
        soup = BeautifulSoup(str(file.read()))
        book_text = clean(soup.get_text())
    row = [name, author, book_text, genre]
    dataset.append(row)
df = pd.DataFrame(dataset, columns=['name', 'author', 'text', 'genre'])
pd.to_pickle(df, 'dataset.pkl')

996it [09:00,  1.84it/s]


In [13]:
df = pd.read_pickle('dataset.pkl')

,name,author,text,genre
0,The Mystery of the Boule Cabinet: A Detective ...,Stevenson| Burton Egbert,detective story b fellow sherlockian hello sai...,Detective and Mystery
1,The Pupil,James| Henry,edition first published text follows definitiv...,Literary
2,At Love's Cost,Garvice| Charles,moment never fully realised great ass man thin...,Literary
3,The Heart of the Range,White| William Patterson,rider golden bar hidden trails lynch lawyers o...,Western Stories
4,The Worshipper of the Image,Gallienne| Richard Le,worshipper image evening wood still dreaming b...,Literary
...,...,...,...,...
991,David Copperfield,Dickens| Charles,find easy get sufficiently far away book first...,Literary
992,Hard Times,Dickens| Charles,one thing needful murdering innocents loophole...,Literary
993,Memoirs of Shelock Holmes,Connan| Doyle,afraid watson shall go said holmes sat togethe...,Detective and Mystery
994,The Mysterious Affair at Styles,Christie| Agatha,intense interest aroused public known time sty...,Detective and Mystery


In [8]:
for i, r in tqdm(df.iterrows()):
    x = r['text']
    
    chunk_len = 10000
    book_len = len(x)
    rem = book_len % chunk_len
    x += x[:(chunk_len - rem)]
    
    a = []
    
    for j in range(10000,len(x)+1,10000):
        tokens=nltk.word_tokenize(x[:j])
        types=nltk.Counter(tokens)
        if(len(tokens) <= 0):
            TTR = 0
        else:
            TTR= len(types)/len(tokens)
        a.append(TTR)
    df.loc[i, "TTR"] = np.average(TTR)

996it [1:04:20,  3.88s/it]


In [11]:
f = []
for i, r in tqdm(df.iterrows()):
    f.append([r['name'], r['author'], r['TTR'], r['genre']])
new_df = pd.DataFrame(f, columns=['name', 'author', 'TTR', 'genre'])
pd.to_pickle(new_df, 'ttr.pkl')

996it [00:00, 4680.82it/s]


## For Augmented Data

In [3]:
df = pd.read_pickle("C:\\Users\\Shipra Dureja\\Downloads\\augmented_data_train.pkl")
for i, r in tqdm(df.iterrows()):
    x = r['text']
    review = re.sub('[^a-zA-Z]', ' ', x)
    review = review.lower()
    review = review.split()
    all_stopwords = stopwords.words('english')
    review = [word for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    chunk_len = 10000
    book_len = len(review)
    rem = book_len % chunk_len
    review += review[:(chunk_len - rem)]
    
    a = []
    
    for j in range(10000,len(review)+1,10000):
        tokens=nltk.word_tokenize(review[:j])
        types=nltk.Counter(tokens)
        if(len(tokens) <= 0):
            TTR = 0
        else:
            TTR= len(types)/len(tokens)
        a.append(TTR)
    # print(TTR)
    df.loc[i, "TTR"] = np.average(TTR)

28it [00:48,  1.72s/it]


In [5]:
f = []
for i, r in tqdm(df.iterrows()):
    f.append([r['name'], r['author'], r['TTR'], r['genre']])
new_df = pd.DataFrame(f, columns=['name', 'author', 'TTR', 'genre'])
pd.to_pickle(new_df, 'augmented_ttr.pkl')

28it [00:00, 3469.44it/s]
